In [1]:
import tables
import os
import string
import glob
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.mlab as mlab
import json
import tensorflow as tf

In [2]:
X_all = np.loadtxt('training_data/A_features.csv', delimiter=',')
y_all = np.array(np.loadtxt('training_data/A_labels.csv', delimiter=','), dtype=np.int)

In [3]:
from sklearn.model_selection import train_test_split
import time

X_not_test, X_test, y_not_rest, y_test = train_test_split(
    X_all, y_all, train_size=.8, random_state=round(time.time()))

X_train, X_val, y_train, y_val = train_test_split(
    X_not_test, y_not_rest, train_size=.85, random_state=round(time.time()))

print(X_train.shape)
print(y_train.shape)
print(X_val.shape)
print(y_val.shape)
print(X_test.shape)
print(y_test.shape)

(1862, 8400)
(1862, 100)
(329, 8400)
(329, 100)
(548, 8400)
(548, 100)


/Library/Frameworks/Python.framework/Versions/3.5/lib/python3.5/site-packages/sklearn/model_selection/_split.py:2026: FutureWarning: From version 0.21, test_size will always complement train_size unless both are specified.
  FutureWarning)


In [4]:
learning_rate = 0.005
epochs = 40
batch_size = 32

# Network Parameters
n_input = 12 * 700
n_classes = 100
dropout = .8 # Dropout, probability to keep units

# 1. Define Variables and Placeholders
x = tf.placeholder(tf.float32, [None, n_input])
y = tf.placeholder(tf.float32, [None, n_classes])

keep_prob = tf.placeholder(tf.float32) #dropout (keep probability)

def build_model(x, dropout, activation):
    
    x = tf.reshape(x, shape=[-1, 12, 700, 1])
    
    conv1 = tf.layers.conv2d(x, filters=16, kernel_size=(3, 3), activation=activation)
    conv1 = tf.layers.max_pooling2d(conv1, 2, 2)

    conv2 = tf.layers.conv2d(conv1, 32, kernel_size=(3, 3), activation=activation)
    conv2 = tf.layers.max_pooling2d(conv2, 2, 2)
    
    fc1 = tf.contrib.layers.flatten(conv2)

    fc1 = tf.layers.dense(fc1, 128, activation=activation)

    fc2 = tf.layers.dense(fc1, 64, activation=activation)
    
    out = tf.layers.dense(fc2, n_classes, activation=tf.sigmoid)

    return out

predictions = build_model(x, keep_prob, activation=tf.nn.relu)
# 3. Define the loss function
with tf.name_scope("loss"):
    loss = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=predictions, labels=y), name='loss')
    tf.summary.scalar("loss", loss)
    

# 4. Define the accuracy
with tf.name_scope("auc_score"):
    auc, update_op = tf.contrib.metrics.streaming_auc(labels=y, predictions=predictions)
    tf.summary.scalar("auc_score", auc)

summary = tf.summary.merge_all()
# 5. Define an optimizer
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(loss)

def feed_next_batch(train_size, batch_size=64):
    
    start = 0
    while start < train_size:
        yield start, start + batch_size
        start += batch_size

In [5]:
sess = tf.Session()
sess.run(tf.global_variables_initializer())
sess.run(tf.local_variables_initializer())

RUN_NAME = 'audio_nn_1st_run'
writer_train = tf.summary.FileWriter('./log/' + RUN_NAME + '/train', graph=sess.graph)
writer_test = tf.summary.FileWriter('./log/' + RUN_NAME + '/test', graph=sess.graph)

steps = 0
# Keep training until reach max iterations
for epoch in range(epochs):

    indices = np.arange(len(y_train))
    np.random.shuffle(indices)
    X_train, y_train = X_train[indices], y_train[indices]

    for start, end in feed_next_batch(len(X_train), batch_size=batch_size):
        # Run optimization op (backprop)
        batch_x, batch_y = X_train[start:end], y_train[start:end]
        sess.run(optimizer, feed_dict={x: batch_x, y: batch_y, keep_prob: dropout})
        
        steps += 1
        
        if steps % 20 == 0:

            # Calculate batch loss and accuracy
            train_auc, _ = sess.run([auc, update_op], feed_dict={x: X_train, y: y_train})
            train_loss, summary_train = sess.run([loss, summary], 
                                                   feed_dict={x: X_train, y: y_train, keep_prob: 1.0})
            writer_train.add_summary(summary_train, steps)

            print('Step %d' % (steps))
            print ("Training loss:", train_loss, ', training auc: ', train_auc)
            
            val_auc, _ = sess.run([auc, update_op], feed_dict={x: X_val, y: y_val})
            val_loss, summary_test = sess.run([loss, summary], feed_dict={x: X_val, 
                                                                        y: y_val,
                                                                       keep_prob: 1.0})
            writer_test.add_summary(summary_test, steps)
            print ("Validation loss:", val_loss, ', validation auc: ', val_auc)
            print('-' * 80)

print ("Optimization Finished!")

# Calculate accuracy for all test samples
# print ("Training AUC:", \
#     sess.run(auc_score, feed_dict={x: X_train,
#                                   y: y_train,
#                                  keep_prob: 1.0}))
# print ("Test AUC:", \
#     sess.run(auc_score, feed_dict={x: X_test,
#                                   y: y_test,
#                                  keep_prob: 1.0}))

Step 20
Training loss: 0.755779 , training auc:  0.0
Validation loss: 0.755613 , validation auc:  0.502582
--------------------------------------------------------------------------------
Step 40
Training loss: 0.73851 , training auc:  0.502231
Validation loss: 0.738678 , validation auc:  0.503305
--------------------------------------------------------------------------------


KeyboardInterrupt: 